In [22]:
import math
import os
import re
from collections import Counter
import string


# Pseudo Code

# store the documents as lists of words, one after another, one list per doc

# For each paper source in 20_newsgroups
#     For each paper in the source
#         remove the first 4 lines (lines starting with Newsgroup, document_id, From, Subject)
#         save into dataset (source, paper)
# for each source in dataset
#     split in half, (train, test)

# dataset complete, good to go

def read_20_newsgroups(directory):
    word_counter = Counter()
    dataset = {}
    for curr_dir, classes, files in os.walk(directory):
        #print(f"Reading directory: {curr_dir}, files: {files}, dirs: {classes}")
        curr_class = curr_dir.rsplit('/', 1)[-1]
        dataset[curr_class] = []
        for file in files:
            file_path = os.path.join(curr_dir, file)
            # print(f"Reading file: {file_path}")
            read_file = []
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    for line in f:
                        bad = ['Newsgroup','document_id', 'From', 'Subject']
                        if any(line.startswith(badd) for badd in bad):
                            continue
                        
                        # print(f'line: {line.strip()}')  # Process each line here
                        words = line.split()  # Convert to lowercase to count case-insensitively
                        words = [word.lower() for word in words if re.match(r'^[a-zA-Z]+$', word)]
                        word_counter.update(words)  # Update the counter with words from the line
                        read_file.extend(words)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
            dataset[curr_class].append(read_file)
    stop_words = word_counter.most_common(200)
    print(f'{dataset.keys()}')
    del dataset['20_newsgroups']
    for c, files in dataset.items():
        # print(f"Class {c} has {len(files)} files")
        for file in files:
            for word in stop_words:
                
                if word[0] in file:
                    file.remove(word[0])
    C = list(dataset.keys())
    print(f'Classes: {C}')
    print(f'{dataset.keys()=}')
    return C, dataset
C, D = read_20_newsgroups("/home/jems/cmsc422/p1/20_newsgroups")
train, test = {}, {}
# split D in half to get train and test
for c, files in D.items():
    train[c] = files[:len(files)//2]
    test[c] = files[len(files)//2:]

dict_keys(['20_newsgroups', 'sci.crypt', 'sci.space', 'comp.sys.mac.hardware', 'soc.religion.christian', 'talk.religion.misc', 'talk.politics.misc', 'comp.os.ms-windows.misc', 'comp.windows.x', 'misc.forsale', 'comp.sys.ibm.pc.hardware', 'rec.sport.hockey', 'rec.motorcycles', 'comp.graphics', 'rec.sport.baseball', 'sci.electronics', 'sci.med', 'talk.politics.guns', 'talk.politics.mideast', 'alt.atheism', 'rec.autos'])
Classes: ['sci.crypt', 'sci.space', 'comp.sys.mac.hardware', 'soc.religion.christian', 'talk.religion.misc', 'talk.politics.misc', 'comp.os.ms-windows.misc', 'comp.windows.x', 'misc.forsale', 'comp.sys.ibm.pc.hardware', 'rec.sport.hockey', 'rec.motorcycles', 'comp.graphics', 'rec.sport.baseball', 'sci.electronics', 'sci.med', 'talk.politics.guns', 'talk.politics.mideast', 'alt.atheism', 'rec.autos']
dataset.keys()=dict_keys(['sci.crypt', 'sci.space', 'comp.sys.mac.hardware', 'soc.religion.christian', 'talk.religion.misc', 'talk.politics.misc', 'comp.os.ms-windows.misc', '

In [23]:
print(f"Train: {train.keys()}")
print(f"Test: {test.keys()}")

Train: dict_keys(['sci.crypt', 'sci.space', 'comp.sys.mac.hardware', 'soc.religion.christian', 'talk.religion.misc', 'talk.politics.misc', 'comp.os.ms-windows.misc', 'comp.windows.x', 'misc.forsale', 'comp.sys.ibm.pc.hardware', 'rec.sport.hockey', 'rec.motorcycles', 'comp.graphics', 'rec.sport.baseball', 'sci.electronics', 'sci.med', 'talk.politics.guns', 'talk.politics.mideast', 'alt.atheism', 'rec.autos'])
Test: dict_keys(['sci.crypt', 'sci.space', 'comp.sys.mac.hardware', 'soc.religion.christian', 'talk.religion.misc', 'talk.politics.misc', 'comp.os.ms-windows.misc', 'comp.windows.x', 'misc.forsale', 'comp.sys.ibm.pc.hardware', 'rec.sport.hockey', 'rec.motorcycles', 'comp.graphics', 'rec.sport.baseball', 'sci.electronics', 'sci.med', 'talk.politics.guns', 'talk.politics.mideast', 'alt.atheism', 'rec.autos'])


In [24]:
# Pseudo Code
import pandas as pd
import numpy as np
# Naive Bayes Implementations

# returns V, log P(c), log P(w|c)
def train_naive_bayes(D, C): # D is the dataset, C is the classes
    # Initialize the count of each class
    ndoc = sum([len(D[c]) for c in D]) # for class in the dataset count the number of documents and sum them
    logprior = {} # initialize the logprior
    bigdoc = {} # initialize the bigdoc
    V = set() # initialize the vocabulary
    for c, files in D.items():
        for file in files:
            for word in file:
                V.add(word)
    print(f'Vocab: {len(V)}')
    count = pd.DataFrame(np.zeros((len(V), len(C))), columns=C, index=list(V))
    loglikelihood = pd.DataFrame(np.zeros((len(V), len(C))), columns=C, index=list(V))
    print(f'vocab: {list(V)[:20]=}')
    print(f'starting bigdoc')
    for c in C:
        bigdoc[c] = []
        for doc in D[c]:
            bigdoc[c].extend(doc)
        print(f'ended bigdoc for {c}')
        # print(f'{bigdoc[c][0]}')
        word_count = Counter(bigdoc[c])
        for word in V:
            count.loc[word, c] = word_count[word]
            # print(f'ended word count for {c}, word is {word} and word count is {count.loc[word, c]}')
            # print(f'lets see: {word in bigdoc[c]}')
    print(f'starting prob calcs')
    for c in C: # for each class in the classes
        print(f'starting for {c}')
        ndoc_c = len(D[c]) # count the number of documents in the class
        logprior[c] = math.log(ndoc_c / ndoc)
        sumcount = count[c].sum() + 1
        for word in V:
            localsum = count.loc[word, c] + 1
            loglikelihood.loc[word, c] = math.log(localsum/sumcount)
        print(f'ending for {c}')
    print(f'ending word calcs')
    print(f'{count=}, {loglikelihood=}')
    return V, logprior, loglikelihood

V, logprior, loglikelihood = train_naive_bayes(train, C)

Vocab: 50564
vocab: list(V)[:20]=['favourites', 'ess', 'objekte', 'kellner', 'relink', 'campi', 'kyle', 'hosers', 'shootings', 'sibyl', 'implying', 'blurb', 'chessmaster', 'chop', 'enmity', 'righteous', 'precident', 'reasoned', 'mate', 'while']
starting bigdoc
ended bigdoc for sci.crypt
ended bigdoc for sci.space
ended bigdoc for comp.sys.mac.hardware
ended bigdoc for soc.religion.christian
ended bigdoc for talk.religion.misc
ended bigdoc for talk.politics.misc
ended bigdoc for comp.os.ms-windows.misc
ended bigdoc for comp.windows.x
ended bigdoc for misc.forsale
ended bigdoc for comp.sys.ibm.pc.hardware
ended bigdoc for rec.sport.hockey
ended bigdoc for rec.motorcycles
ended bigdoc for comp.graphics
ended bigdoc for rec.sport.baseball
ended bigdoc for sci.electronics
ended bigdoc for sci.med
ended bigdoc for talk.politics.guns
ended bigdoc for talk.politics.mideast
ended bigdoc for alt.atheism
ended bigdoc for rec.autos
starting prob calcs
starting for sci.crypt
ending for sci.crypt
st

In [25]:
print(f'{loglikelihood=}')

loglikelihood=            sci.crypt  sci.space  comp.sys.mac.hardware  \
favourites -11.554077 -11.216499             -10.808616   
ess        -11.554077 -11.216499             -10.808616   
objekte    -11.554077 -11.216499             -10.808616   
kellner    -11.554077 -11.216499             -10.808616   
relink     -11.554077 -11.216499             -10.808616   
...               ...        ...                    ...   
molecular  -11.554077  -9.019274             -10.115469   
glutamate  -11.554077 -11.216499             -10.808616   
involoved  -11.554077 -11.216499             -10.808616   
telephone   -7.703929  -9.270588              -9.199178   
dn         -10.860930  -9.607061             -10.808616   

            soc.religion.christian  talk.religion.misc  talk.politics.misc  \
favourites              -11.700085          -11.396897          -11.770702   
ess                     -11.700085          -11.396897          -11.770702   
objekte                 -11.700085         

In [35]:
def test_naive_bayes(testdoc, C, V, logprior, loglikelihood):
    summ = logprior.copy()
    #print(f'logprior: {logprior=}')
    for c in C:
        for word in testdoc:
            if word in V:
                #print(f'interesting: {loglikelihood.loc[word, c]}')
                summ[c] = summ[c] + loglikelihood.loc[word, c]
    #print(f'{summ=}')
    total = float('-inf')
    best = None
    for c, val in summ.items():
        if isinstance(val, (int, float)) and val > total:
            best = c
            total = val
    # print(f'{summ=}')
    return best
# c, files = list(test.items())[0]
# file = files[0]
# trial = test_naive_bayes(file, C, V, logprior=logprior, loglikelihood=loglikelihood)
# print(f'{trial=} {c=}')

In [32]:
# def debugging(testset, logprior, likelihood, C, V):
#     counter = 2
#     for c, files in testset.items():
#         for file in files:
#             if counter < 0:
#                 break
#             right = test_naive_bayes(file, C, V, logprior=logprior, loglikelihood=loglikelihood)
#             print(f'result = {right=}, class = {c}')
#             counter -= 1
#         if counter < 0:
#             break
# debugging(testset=test, C=C, V=V, logprior=logprior, likelihood=loglikelihood)

In [37]:
def eval_bayes(testset, logprior, loglikelihood, C, V):
    correct = 0
    total = 0
    #print(f'{dict(list(testset.items())[:2])=}')
    for c, files in testset.items():
        for file in files:
            #print(f'curr file {file=}')
            right = test_naive_bayes(file, C, V, logprior=logprior, loglikelihood=loglikelihood)
            #print(f'the right class: {c}, what we got: {right}')
            if c == right:
                correct += 1
            total += 1
    return correct/total
    

accuracy = eval_bayes(testset=test, C=C, V=V, logprior=logprior, loglikelihood=loglikelihood)
print(accuracy)

0.7606760676067607


In [38]:
print(f'{accuracy=}')

accuracy=0.7606760676067607
